In [1]:
# Taken from GPYTorch Examples

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

import urllib.request
import os
from scipy.io import loadmat
from math import floor
import time

## Data

In [3]:
data = torch.Tensor(loadmat('../elevators.mat/elevators.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

#if torch.cuda.is_available():
#    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

### Models 

In [4]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.base_covar_module = ScaleKernel(RBFKernel())
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[:500, :], likelihood=likelihood)
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [ ]:
class GPRegressionModelHalf(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.base_covar_module = ScaleKernel(RBFKernel())
        self.covar_module = InducingPointKernelHalf(self.base_covar_module, inducing_points=train_x[:500, :], likelihood=likelihood)
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

### Likelihood, MLL and Optimizer

In [5]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

likelihood.train()
model.train()

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
likelihoodhalf = GaussianLikelihoodHalf()
modelhalf = GPRegressionModelHalf(train_x, train_y, likelihood)

likelihoodhalf.train()
modelhalf.train()

mllhalf = ExactMarginalLogLikelihoodHalf(likelihood, model)
optimizerhalf = torch.optim.Adam(model.parameters(), lr=0.01)

### Train

In [6]:
def train():
    start = time.time()
    for i in range(50):
        optimizer.zero_grad()
        output = model(train_x)
        
        #addterm = modelhalf.covar_module._get_added()
        #print('Iter %d/%d - Added Loss Term: %.3f' % (i + 1, 50, addterm.item()))

        loss = -mll(output, train_y)
        loss.backward()
        
        print('Iter %d/%d - Loss: %.3f' % (i + 1, 50, loss.item()))
        optimizer.step()
        torch.cuda.empty_cache()
    
        
def trainhalf():
    starthalf = time.time()
    for i in range(50):
        optimizerhalf.zero_grad()
        outputhalf = modelhalf(train_x)
        
        #addtermhalf = modelhalf.covar_module._get_added()
        #print('Iter %d/%d - Half Added Loss Term: %.3f' % (i + 1, 50, addtermhalf.item()))
        
        losshalf = -mllhalf(outputhalf, train_y)
        losshalf.backward()
        
        print('Iter %d/%d - Loss: %.3f' % (i + 1, 50, losshalf.item()))
        optimizerhalf.step()
        torch.cuda.empty_cache()

### Test

In [7]:
model.eval()
likelihood.eval()
with gpytorch.settings.max_preconditioner_size(10), torch.no_grad():
    preds = model(test_x)

C:\Users\PaulQuantum\Anaconda3\lib\site-packages\gpytorch\kernels\inducing_point_kernel.py:61: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  inv_root = torch.triangular_solve(eye, chol)[0]


In [ ]:
modelhalf.eval()
likelihoodhalf.eval()
with gpytorch.settings.max_preconditioner_size(10), torch.no_grad():
    predshalf = modelhalf(test_x)

In [18]:
print('Test MAE Full: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))
print('Test MAE Half: {}'.format(torch.mean(torch.abs(predshalf.mean - test_y))))

Test MAE: 0.0731787383556366


### Properties (TODO) 

In [ ]:
print('KL Div Term Full: {}'.format(addterm))
print('KL Div Term Half: {}'.format(addtermhalf))